In [32]:
using LightGraphs, GraphIO
using TimerOutputs
using StatsBase

In [33]:
const to = TimerOutput()
dir = "input_files"
input_files = filter(x -> isfile(joinpath(dir, x)), readdir(dir))

6-element Array{String,1}:
 "airtc.txt"
 "dolphins.txt"
 "facebook_combined.txt"
 "karateclub.txt"
 "lesmis.txt"
 "mcldata.txt"

In [54]:
function label_partition(g; k=8)
  unlabeled_nodes = Set(vertices(g));
  labeled_nodes = Set();
  finished_nodes = Set();
  labels = zeros(Int, nv(g));
  random_k_nodes = sample(vertices(g), k, replace=false);
  for (i, r) in enumerate(random_k_nodes)
    labels[r] = i;
    unlabeled_nodes = delete!(unlabeled_nodes, r)
    labeled_nodes = push!(labeled_nodes, r)
  end
  
  operations = 0
  while !isempty(unlabeled_nodes)
    for n in copy(labeled_nodes)
      n_neighbors = setdiff(Set(neighbors(g, n)), union(labeled_nodes, finished_nodes))
      operations += 1
      if !isempty(n_neighbors)
        neighbor = sample(collect(n_neighbors), 1)[1];
        labels[neighbor] = labels[n];
        unlabeled_nodes = delete!(unlabeled_nodes, neighbor);
        labeled_nodes = push!(labeled_nodes, neighbor);
        n_neighbors = delete!(n_neighbors, neighbor);
      end
      if isempty(n_neighbors)
        labeled_nodes = delete!(labeled_nodes, n);
        finished_nodes = push!(finished_nodes, n);
      end
    end
  end
  return labels
end

function graph_partition(g; k=8, c=10)
  best_partition = []
  best_modularity = -Inf
  for i in 1:c
    partitions = label_partition(g; k=k)
    partition_modularity = modularity(g, partitions)
#     println(partition_modularity)
    if (best_modularity < partition_modularity)
      best_partition = partitions
      best_modularity = partition_modularity
    end
  end
  return best_partition, best_modularity
end

graph_partition (generic function with 1 method)

In [51]:
filename = joinpath(dir, input_files[2])
println(filename)
g = SimpleGraph(loadgraph(filename, "graph", GraphIO.EdgeList.EdgeListFormat()))

input_files\dolphins.txt


{62, 159} undirected simple Int64 graph

In [42]:
best_modularity = -Inf
best_k = 0
for k in 2:15
  @timeit to filename part, modu = graph_partition(g; k=k, c=25)
  if (best_modularity < modu)
    best_modularity = modu
    best_k = k
  end
end
round(best_modularity, digits=4), best_k

(0.365, 4)

In [55]:
@timeit to filename part, modu = graph_partition(g; k=8, c=25)

([7, 4, 4, 1, 1, 4, 7, 6, 4, 3  …  1, 7, 5, 3, 4, 4, 5, 3, 4, 7], 0.38036470076341916)

In [44]:
to

 ──────────────────────────────────────────────────────────────────────────────
                                       Time                   Allocations      
                               ──────────────────────   ───────────────────────
       Tot / % measured:            67.7s / 0.38%            305MiB / 30.9%    

 Section               ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────────────────
 input_files\karate...     18    258ms   100%  14.3ms   94.5MiB  100%   5.25MiB
 ──────────────────────────────────────────────────────────────────────────────

In [56]:
d = []
for i in 1:20
  par, op = label_partition(g; k=i)
  push!(d, op)
end
println(nv(g))
d

62


20-element Array{Any,1}:
  1
  1
  2
  4
  5
  5
  2
  8
  2
 10
 10
 11
 13
 14
 15
  4
 15
 14
 19
 10

In [31]:
34 * 2

68